# Installation & Imports

In [ ]:
#install
!pip install transformers==4.28.0  # due to issue https://github.com/huggingface/transformers/issues/22816
!pip install datasets

!pip install evaluate
!apt install git-lfs
import transformers
print(transformers.__version__)
from google.colab import drive
from transformers import AutoTokenizer, DataCollatorWithPadding, pipeline
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import datasets
import evaluate
import os


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.0 MB/s eta 0:00:0

In [ ]:
drive.mount('/content/gdrive/', force_remount=True)

data_dir = '/content/gdrive/path/to/data'

# input file: hydrated Tweets
# format: parquet
# columns: tweet, is_misinfo
# index = id
ds2_fp = os.path.join(data_dir, 'antivax.parquet')

# output dirs
dataset_dir = os.path.join(data_dir, 'vaccine_ds')
model_dir = os.path.join(data_dir, 'vaxx_bert_model')
os.path.exists(data_dir )

Mounted at /content/gdrive/


True

# Load Dataset

In [ ]:
col_text, col_label, col_label_orig = 'after_lemmatization', 'label', 'is_misinfo'  # 'tweet'
cols_orig = [col_text, col_label_orig]

In [ ]:
from datasets import DatasetDict, Dataset
def create_dataset():
  ds = DatasetDict()
  df = pd.read_parquet(ds2_fp)
  df = df[cols_orig]
  df = df[~df.index.duplicated(keep='first')]
  df.rename(columns={col_label_orig:col_label}, inplace=True)

  print(df.head())
  print('label values', df[col_label].value_counts())

  df1 = df[df[col_label] == 0]
  df2 = df[df[col_label] == 1]
  assert len(df1.index) > .3 * len(df2.index)
  #breakpoint()
  train_frac = .85
  df1i = df1.sample(frac=train_frac, random_state=1)
  df2i = df2.sample(frac=train_frac, random_state=1)
  df_train = pd.concat((df1i, df2i), axis=0)
  df_test = df[~df.index.isin(df_train.index)]
  print('train labels', df_train[col_label].value_counts())
  print('test labels', df_test[col_label].value_counts())
  ds['train'] = Dataset.from_pandas(df_train)
  ds['test'] = Dataset.from_pandas(df_test)
  return ds

In [ ]:
# dataset format:
# {"test": [{"label": 0, "text": "blabla"}, …], "train: …"
import pandas as pd

ds = create_dataset()
ds.save_to_disk(dataset_dir)
ds = ds.shuffle(seed=42)
print(ds)
print('train', ds['train'][0])
print('train', ds['train'][-1])
print('test', ds['test'][-1])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples[col_text], truncation=True)

tokenized_ds = ds.map(preprocess_function, batched=True) # same as spencer

Map:   0%|          | 0/11990 [00:00<?, ? examples/s]

Map:   0%|          | 0/2116 [00:00<?, ? examples/s]

In [ ]:
# efficient to dynamically pad the sentences to the longest length in a batch during collation
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # same as spencer

# Evaluate

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# Train

In [ ]:
id2label = {0: "no_misinfo", 1: "is_misinfo"}  # or LABEL_0 and LABEL_1
label2id = {"no_misinfo": 0, "is_misinfo": 1}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifi

In [ ]:
#!pip install --upgrade accelerate

model for corona vaccines https://drive.google.com/file/d/1QW9D6TN4KXX6poa6Q5L6FVgqaDQ4DxY9/view
uses 3 epochs and 4392 optimization steps.

Parameters from https://huggingface.co/docs/transformers/v4.17.0/en/custom_datasets, I use 3 instead of 5 epochs

spencer uses arguments as here: training_args = TrainingArguments("test-trainer")

In [ ]:
print(TrainingArguments("test-trainer"))

TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignor

In [ ]:
training_args = TrainingArguments(
    output_dir=model_dir,
    learning_rate=2e-5,  #default 5e-05,
    per_device_train_batch_size=16,  # default 8
    per_device_eval_batch_size=16,  # default 8
    num_train_epochs=3, # default = 3, examples uses 5
    weight_decay=0.01,  # default = 0
    evaluation_strategy="steps",  #default =no,
    eval_steps=500,  # default None
    load_best_model_at_end=True,
    save_total_limit=2,  # default None
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train() # resume_from_checkpoint = True -> change path above
trainer.save_model() # saves best model to model_dir, Means that the last saved checkpoint is the checkpoint with the best validation score instead of the final weights if I am saving multiple checkpoints

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss,Accuracy
500,0.186500,0.150068,0.960775
1000,0.080900,0.154993,0.959357
1500,0.069100,0.163421,0.963611
2000,0.034900,0.174047,0.962193


# Use it

In [ ]:

text =  'RT: Vaccine kills people!'
classifier = pipeline("sentiment-analysis", model=model_dir)
classifier(text)